In [1]:
import fastai2
from fastai2.tabular.all import *
from fastai2.metrics import *

In [2]:
from sklearn.model_selection import train_test_split

In [3]:
from copy import deepcopy

In [4]:
import optuna

In [5]:
SEED = 42

# HyPSTER Classes

In [6]:
from inspect import signature, Parameter
import functools

def auto_assign(func):
    # Signature:
    sig = signature(func)
    for name, param in sig.parameters.items():
        if param.kind in (Parameter.VAR_POSITIONAL, Parameter.VAR_KEYWORD):
            raise RuntimeError('Unable to auto assign if *args or **kwargs in signature.')
    # Wrapper:
    @functools.wraps(func)
    def wrapper(self, *args, **kwargs):
        for i, (name, param) in enumerate(sig.parameters.items()):
            # Skip 'self' param:
            if i == 0: continue
            # Search value in args, kwargs or defaults:
            if i - 1 < len(args):
                val = args[i - 1]
            elif name in kwargs:
                val = kwargs[name]
            else:
                val = param.default
            setattr(self, name, val)
        func(self, *args, **kwargs)
    return wrapper

In [21]:
def sample_hp(hp, trial): return hp if not isinstance(hp, HpExpression) else hp.sample(trial)

In [22]:
x = "d"

In [25]:
len(x) + 1

2

In [26]:
class HpExpression(object):
    def __init__(self, exp1, exp2):
        self.exp1 = exp1; self.exp2 = exp2    
    
    def sample(self, trial): raise NotImplementedError
    
    def get_name(self):
        name = ""
        if self.exp1 is not None and isinstance(self.exp1, HpExpression) and self.exp1.name is not None:
            name += self.exp1.name
        if self.exp2 is not None and isinstance(self.exp2, HpExpression) and self.exp2.name is not None:
            if len(name) > 0:
                name +=  "_"
            name += self.exp2.name
        self.name = name
        return self.name
        #TODO: refactor
        
    def __add__(self, other):  return AddExpression(self, other)
    def __radd__(self, other): return AddExpression(other, self)
    def __sub__(self, other):  return SubExpression(self, other)
    def __rsub__(self, other): return SubExpression(other, self)
    def __mul__(self, other):  return MulExpression(self, other)
    def __rmul__(self, other): return MulExpression(other, self)
    def __div__(self, other):  return DivExpression(self, other)
    def __rdiv__(self, other): return DivExpression(other, self)
    def __pow__(self, other):  return PowExpression(self, other)
    def __rpow__(self, other): return PowExpression(other, self)

In [27]:
class SubExpression(HpExpression):
    def sample(self, trial):
        exp1 = sample_hp(self.exp1, trial)
        exp2 = sample_hp(self.exp2, trial)
        return exp1 - exp2

In [28]:
class AddExpression(HpExpression):    
    def sample(self, trial):
        exp1 = sample_hp(self.exp1, trial)
        exp2 = sample_hp(self.exp2, trial)
        return exp1 + exp2

In [29]:
class MulExpression(HpExpression):    
    def sample(self, trial):
        exp1 = sample_hp(self.exp1, trial)
        exp2 = sample_hp(self.exp2, trial)
        return exp1 * exp2

In [30]:
class DivExpression(HpExpression):    
    def sample(self, trial):
        exp1 = sample_hp(self.exp1, trial)
        exp2 = sample_hp(self.exp2, trial)
        return exp1 / exp2

In [31]:
class PowExpression(HpExpression):    
    def sample(self, trial):
        exp1 = sample_hp(self.exp1, trial)
        exp2 = sample_hp(self.exp2, trial)
        return exp1 ** exp2

In [32]:
#TODO: add round, int, floor etc...
#TODO: add Brackets () etc...?

In [33]:
class HpFloat(HpExpression):
    @auto_assign
    def __init__(self, name, low, high, log=False, step=None):
        self.result = None
        #TODO: check what's up with log and step
        #TODO: move result to HpExpression?
        
    def sample(self, trial): 
        self.result = ifnone(self.result, trial.suggest_float(self.name, self.low, self.high))
        return self.result
    
    #TODO: warn if log=True & step is not None
    #TODO: check what is the "*" in the function definition

In [34]:
class HpInt(HpExpression):
    @auto_assign
    def __init__(self, name, low, high, step=1):
        self.result = None
    
    def sample(self, trial):
        self.result = ifnone(self.result, trial.suggest_int(self.name, self.low, self.high, self.step))
        return self.result

In [35]:
class HpCategorical(HpExpression):
    @auto_assign
    def __init__(self, name, choices): 
        self.result = None
    
    def sample(self, trial): 
        if self.result is not None:
            return self.result
        
        choices           = self.choices
        name              = self.name
        optuna_valid_cats = ["str", "int", "float", "bool"] #TODO: add more + move to global area
        
        if any([type(choice) not in optuna_valid_cats for choice in self.choices]):
            self.dict_keys  = [choice.__name__ for choice in choices]
            self.dict_items = dict(zip(self.dict_keys, choices))
            chosen_hp       = trial.suggest_categorical(name, self.dict_keys)
            self.result     = self.dict_items[chosen_hp]
            #TODO: add items dict
        else:
            self.result = trial.suggest_categorical(name, choices)
        return self.result

In [36]:
class HpList(HpExpression):
    @auto_assign
    def __init__(self, name, min_len, max_len, hp, same_value=False): pass        
    
    def sample(self, trial):
        lst_len = trial.suggest_int(self.name, self.min_len, self.max_len)
        lst = []
        if (self.same_value) or (not isinstance(self.hp, HpExpression)):
            lst = [sample_hp(self.hp, trial)] * lst_len
        else:
            for i in range(lst_len):
                hp = deepcopy(self.hp)
                hp.name = f"{hp.get_name()}_{i+1}"
                result = sample_hp(hp, trial)
                lst.append(result)
                #TODO keep self.result?
        return lst

In [37]:
class HpIterable(HpExpression):
    def __init__(self, name, iterable): 
        self.name = name
        self.iterable = iterable

    def _sample_list(self, trial, lst):  return [sample_hp(item, trial) for item in lst]
    def _sample_dict(self, trial, dct):  return {key : sample_hp(value, trial) for key, value in dct.items()}
    def _sample_tuple(self, trial, tup): return (*self._sample_list(trial, tup), )
    def _sample_L(self, trial, l):       return L(self._sample_list(trial, l))
    
    def sample(self, trial):
        if   isinstance(self.iterable, dict):   return self._sample_dict(trial, self.iterable)
        elif isinstance(self.iterable, list):   return self._sample_list(trial, self.iterable)
        elif isinstance(self.iterable, L):      return self._sample_L(trial, self.iterable)
        elif isinstance(self.iterable, tuple):  return self._sample_tuple(trial, self.iterable)
        else:                                   print("Error: unknown Iterable!")
        return

In [38]:
class HpBool(HpCategorical):
    def __init__(self, name):
        super().__init__(name, choices=[False, True])

In [39]:
class HpToggle(HpBool):
    def __init__(self, hp): return
    def sample(self, trial): return trial.suggest_categorical(f"toggle_{hp.name}", [False, True]) 
    #TODO: fix hp.name

In [40]:
x = HpInt("start_mom", 2, 10)

In [41]:
x = HpList("layers", 1, 5, 50 * HpInt("layer_size", 1, 6), same_value=False)

In [42]:
optuna.logging.set_verbosity(0)
pruner = optuna.pruners.NopPruner()
study = optuna.create_study(direction="maximize", pruner=pruner)

In [43]:
def objective(trial):
    print(x.sample(trial))
    #print(y.sample(trial))
    return 1.0

In [45]:
study.optimize(objective, n_trials=5, timeout=600)

[50, 50, 50]
[100, 100]
[150, 150, 150, 150]
[300, 300, 300, 300]
[200, 200]


In [12]:
#TODO consider adding "name" into HpExpression

In [24]:
#HpFunc

In [25]:
#HpTuple (like list)
#HpIterable (?)

In [26]:
#TODO: support expressions like 5 * HpInt(...)
#TODO: support expressions like HpInt(...) * HpFloat(...)

# Read Data

In [27]:
path = untar_data(URLs.ADULT_SAMPLE)
path.ls()

(#3) [Path('C:/Users/user/.fastai/data/adult_sample/adult.csv'),Path('C:/Users/user/.fastai/data/adult_sample/export.pkl'),Path('C:/Users/user/.fastai/data/adult_sample/models')]

In [28]:
df = pd.read_csv(path/'adult.csv')
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,49,Private,101320,Assoc-acdm,12.0,Married-civ-spouse,NaN,Wife,White,Female,0,1902,40,United-States,>=50k
1,44,Private,236746,Masters,14.0,Divorced,Exec-managerial,Not-in-family,White,Male,10520,0,45,United-States,>=50k
2,38,Private,96185,HS-grad,NaN,Divorced,NaN,Unmarried,Black,Female,0,0,32,United-States,<50k
3,38,Self-emp-inc,112847,Prof-school,15.0,Married-civ-spouse,Prof-specialty,Husband,Asian-Pac-Islander,Male,0,0,40,United-States,>=50k
4,42,Self-emp-not-inc,82297,7th-8th,NaN,Married-civ-spouse,Other-service,Wife,Black,Female,0,0,50,United-States,<50k


In [29]:
df = df.sample(frac=0.3)

In [30]:
cat_names = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race']

In [31]:
cont_names = ['age', 'fnlwgt', 'education-num']

In [32]:
dep_var = "salary"

In [33]:
train_df, test_df = train_test_split(df, test_size=0.6, 
                                     random_state=SEED, 
                                     stratify=df[dep_var])

# Preprocessing

In [34]:
cat = Categorify()

In [35]:
imp = FillMissing(fill_strategy=FillStrategy.mode, 
                  add_col=True)

In [36]:
norm = Normalize()

In [37]:
procs = [cat, imp, norm]

# DataBunch

In [38]:
to = TabularPandas(train_df, 
                   y_block = CategoryBlock(), 
                   y_names = dep_var,
                   splits = RandomSplitter()(range_of(train_df)),
                   cat_names = cat_names,
                   cont_names = cont_names,
                   procs = procs)

In [39]:
dls = to.dataloaders(batch_size=32)

# Learner

In [40]:
cbs = [TrackerCallback(monitor="roc_auc_score"), ReduceLROnPlateau("roc_auc_score", patience=3)]

In [41]:
class HypsterBase(object):
    def __init__(self): return

In [42]:
class HypsterTabularLearner(HypsterBase):
    def __init__(self, dls, **kwargs):
        self.dls = dls
        self.__dict__.update(kwargs)
        self.func = raw_tabular_learner

In [43]:
raw_tabular_learner = fastai2.tabular.learner.tabular_learner

In [44]:
hypster_types = (HypsterBase, HpExpression)

In [45]:
data_structures = (set, list, dict, L)

In [46]:
def _contains_hypster(x):
    if not isinstance(x, data_structures):
        x = [x]
    elif isinstance(x, dict):
        x = x.values()
        
    for item in x:
        if isinstance(item, data_structures):
            if _iterable_contains_hypster(item):
                return True
        else:
            if isinstance(item, hypster_types):
                return True
    return False

In [47]:
@delegates(raw_tabular_learner)
def tabular_learner(dls, **kwargs):
    kwargs["dls"] = dls
    for key, value in kwargs.items():
        if _contains_hypster(value):
            return HypsterTabularLearner(**kwargs)
    return raw_tabular_learner(**kwargs)

In [48]:
#def tabular_learner():
    #put arguments in func definition
    #add state to learner = superposition or not
    #add trial/study
    #check if opt_func is type list
        #if so - randomly choose one
    #call function

In [49]:
cbs = [TrackerCallback(monitor="roc_auc_score"), 
       HpToggle(ReduceLROnPlateau("roc_auc_score", patience=HpInt("patience", 1, 5)))]

In [50]:
cbs = [TrackerCallback(monitor="roc_auc_score"), 
       ReduceLROnPlateau("roc_auc_score", patience=1)
      ]

In [51]:
start_mom = HpFloat("start_mom", 0.85, 0.99)

In [52]:
learn = tabular_learner(dls=dls, 
                        metrics=RocAuc(),
                        layers=HpList("layers", 1, 3, 50 * HpInt("layer_size", 1, 6), same_value=False),
                        opt_func=HpCategorical("optimizer", [Adam, SGD, QHAdam]),
                        cbs=cbs,
                        moms=(start_mom, start_mom-0.1, start_mom), 
                        #wd_bn_bias=HpBool("wd_bn_bias"), 
                        )

# Optuna

In [53]:
class HypsterExperiment(HypsterBase):
    def __init__(self, learner):
        self.learner = learner
    def fit(self, n_trials):
        self.study = study_fit(self.learner, n_trials)        

In [54]:
from inspect import signature

In [55]:
def get_func_args(sig):
    args = []
    for arg, value in sig.parameters.items():
        if not "=" in str(value):
            args.append(arg)
    return args

In [56]:
def pop_move_dict(dict1, keys):
    lst = []
    for key in list(dict1):
        if key in keys:
            lst.append(dict1.pop(key))
    return lst

In [61]:
class Objective(object):
    def __init__(self, learner):
        self.learn = learner
    def __call__(self, trial):
        learner = deepcopy(self.learn)
        func = learner.__dict__.pop("func")
        actual_dict = {}
        for key, value in learner.__dict__.items():
            if _contains_hypster(value):
                if isinstance(value, data_structures):
                    
                else:
                    actual_dict[key] = value.sample(trial)
            else:
                actual_dict[key] = value
        
        sig = signature(func)
        func_args = get_func_args(sig)
        args = pop_move_dict(actual_dict, func_args)
        print(args)
        print("\n")
        print(actual_dict)
        actual_learner = func(*args, **actual_dict)
        actual_learner.fit_one_cycle(2)
        print(actual_learner.cbs)
        return actual_learner.cbs[3].best

In [62]:
def study_fit(learner, n_trials):
    optuna.logging.set_verbosity(0)
    pruner = optuna.pruners.NopPruner()
    study = optuna.create_study(direction="maximize", pruner=pruner)
    objective = Objective(learner)
    study.optimize(objective, n_trials=n_trials, timeout=600)
    return study

In [63]:
clf = HypsterExperiment(learn)

In [64]:
clf.fit(n_trials=2)



{'metrics': <fastai2.metrics.AccumMetric object at 0x000002AA57C3C978>, 'layers': [250, 250], 'opt_func': <function Adam at 0x000002AA532F3158>, 'cbs': [TrackerCallback, ReduceLROnPlateau], 'moms': (<__main__.HpFloat object at 0x000002AA533CE710>, <__main__.HpFloat object at 0x000002AA533CE710>, <__main__.HpFloat object at 0x000002AA533CE710>)}


[W 2020-04-24 22:49:48,321] Setting status of trial#0 as TrialState.FAIL because of the following error: IndexError('list index out of range')
Traceback (most recent call last):
  File "C:\Users\user\Anaconda3\lib\site-packages\fastai2\learner.py", line 192, in fit
    self._do_epoch_train()
  File "C:\Users\user\Anaconda3\lib\site-packages\fastai2\learner.py", line 165, in _do_epoch_train
    self.all_batches()
  File "C:\Users\user\Anaconda3\lib\site-packages\fastai2\learner.py", line 143, in all_batches
    for o in enumerate(self.dl): self.one_batch(*o)
  File "C:\Users\user\Anaconda3\lib\site-packages\fastai2\learner.py", line 148, in one_batch
    self._split(b);                                  self('begin_batch')
  File "C:\Users\user\Anaconda3\lib\site-packages\fastai2\learner.py", line 124, in __call__
    def __call__(self, event_name): L(event_name).map(self._call_one)
  File "C:\Users\user\Anaconda3\lib\site-packages\fastcore\foundation.py", line 372, in map
    return sel

IndexError: list index out of range

In [ ]:
clf.study.trials_dataframe()

In [ ]:
#learn = ...
hps = HypsterExperiment(learn, learn.fit_one_cycle, n_trials=3) 
hps.fit()

In [84]:
import optuna

In [142]:
EPOCHS = 2

In [149]:
def objective(trial):
    #Q: How do I define fill value?
    cat = Categorify()
    fillstraFillStrategy.mode
    imp = FillMissing(fill_strategy=fill_strategy,
                      add_col=True)

    norm = Normalize(mean=5)
    procs = [cat, imp, norm]

    # DataBunch
    to = TabularPandas(train_df, 
                       y_block = CategoryBlock(), 
                       y_names = dep_var,
                       splits = RandomSplitter()(range_of(train_df)),
                       cat_names = cat_names,
                       cont_names = cont_names,
                       procs = procs)

    dls = to.dataloaders(batch_size=512)
    
    n_layers = trial.suggest_int("n_layers", 2, 5)
    layer_sizes = L()
    
    for i in range(n_layers):
        layer_size = trial.suggest_int("layer_size_{}".format(i), 1, 10)
        layer_sizes.append(50 * layer_size)

    learn = tabular_learner(dls, metrics=RocAuc(),
                            layers=layer_sizes, 
                            #emb_szs=[], 
                            #loss_func,
                            #opt_func, 
                            cbs=cbs,
                            #lr=0.001, moms=(0.95, 0.85, 0.95)
                            #wd=None, wd_bn_bias=False, train_bn=True
                           )
    
    learn.fit_flat_cos(EPOCHS)

    return learn.cbs[3].best

In [150]:
optuna.logging.set_verbosity(0)
pruner = optuna.pruners.NopPruner()
study = optuna.create_study(direction="maximize", pruner=pruner)
study.optimize(objective, n_trials=5, timeout=600)

epoch,train_loss,valid_loss,roc_auc_score,time
0,0.405053,0.476074,0.500000,00:01
1,0.367246,0.395882,0.622816,00:01


epoch,train_loss,valid_loss,roc_auc_score,time
0,0.380378,0.460215,0.538600,00:02
1,0.356267,0.383154,0.686228,00:02


epoch,train_loss,valid_loss,roc_auc_score,time
0,0.383377,0.471449,0.507842,00:01
1,0.358828,0.389051,0.687395,00:01


epoch,train_loss,valid_loss,roc_auc_score,time
0,0.394768,0.472647,0.671344,00:01
1,0.363715,0.388233,0.732455,00:01


epoch,train_loss,valid_loss,roc_auc_score,time
0,0.383316,0.457935,0.591756,00:01
1,0.359925,0.386684,0.720578,00:01


In [151]:
print("Number of finished trials: {}".format(len(study.trials)))

Number of finished trials: 5


In [152]:
print("Best trial:")
trial = study.best_trial
print("  Value: {}".format(trial.value))

Best trial:
  Value: 0.7324549813617556


In [153]:
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

  Params: 
    n_layers: 2
    layer_size_0: 7
    layer_size_1: 3


# Test Results

In [154]:
test_dl = learn.dls.test_dl(test_df.drop(['salary'], axis=1))

In [155]:
probs = learn.get_preds(dl=test_dl)

In [156]:
probs = probs[0][:,1]

In [157]:
from sklearn.metrics import roc_auc_score

In [158]:
scorer = RocAuc()

In [159]:
roc_auc_score(test_df[dep_var], probs)

0.8787596624870792

# Desired API

In [160]:
import numpy as np

## Preprocessing

In [ ]:
fill = HpInt(start_range=1, end_range=100)

In [ ]:
fill_strategy = HpOptions("fill_strategy", 
                          [FillStrategy.mode, 
                           FillStrategy.median, 
                           FillStrategy.constant(5, fill)])

In [ ]:
# or less preferred

In [ ]:
fill_dict = {"mode" : FillStrategy.mode, "median" : FillStrategy.median, "constant" : FillStrategy.constant(5, fill)}
fill_strategy = HpOptions("fill_strategy", ["mode", "median", "constant"], fill_dict)

In [ ]:
add_col = HpBool("missing_col_bool")

In [64]:
imp = FillMissing(fill_strategy=fill_strategy, add_col=add_col)

#### Option A

In [65]:
norm = Normalize(mean=HpFloat("norm_mean", start=2, end=10, dist="uniform"))

In [66]:
procs = [Categorify, imp, HpToggle(norm)]

#### Option B

In [ ]:
norm = HpToggle(Normalize(mean=HpFloat(start=2, end=10, dist="uniform")))

In [82]:
procs = [Categorify, imp, norm]

## DataBunch

In [67]:
to = TabularPandas(train_df, 
                   y_block = CategoryBlock(), 
                   y_names = dep_var,
                   splits = RandomSplitter()(range_of(train_df)),
                   cat_names = cat_names,
                   cont_names = cont_names,
                   procs = procs)

#### Option A

In [ ]:
bs_pow = HpInt(0, 8)

In [122]:
dls = to.dataloaders(batch_size=2**bs_pow)
#or
dls = to.dataloaders(batch_size=HpConst(2)**bs_pow)

#### Option B

In [122]:
dls = to.dataloaders(batch_size=HpBatchSizeFinder(...))

## Learner

In [123]:
from fastai2.metrics import *

In [124]:
cbs = [TrackerCallback(monitor="roc_auc_score")]

### #Layers + Layer Sizes

#### Option A

In [ ]:
layer_size_hp = HpFuncInt(func=np.multiply, base_value=50, min_int=1, max_int=7)
layers = HpVarList(min_len=1, max_len=5, layer_size_hp)

#### Option B

In [ ]:
layer_size_hp = HpInt(min=1, max=7)
layers = HpVarList(min_len=1, max_len=5, value=50 * layer_size_hp) 
#TODO: think of how to distinguish between same value for all items in list and different ones?

### Optimizer

In [ ]:
opt_dict = {"SGD" : SGD, "ADAM" : Adam, "LAMB" : fastai2.optimizer.Lamb()}

In [ ]:
optimizer = HpOptions(["SGD", "ADAM", "LAMB"], opt_dict)

In [ ]:
if optimizer == "SGD":
    sqr_mom = Adam()
    Lamb(sqr_mom)

#### Option B

In [ ]:
def opt_name_to_opt(name): if name.containts("SGD") return SGD else Adam

In [ ]:
optimizer = HpOptions(["SGD", "ADAM", "LAMB"], opt_name_to_opt)

#### Option C

In [ ]:
optimizer = HpOptions([SGD, Adam, Lamb])

### Init Learner

In [ ]:
learn = tabular_learner(dls, metrics=RocAuc(),
                        layers=layers
                        
                        #loss_func,
                        opt_func = opt_func
                        cbs=cbs,
                        #moms=(0.95, 0.85, 0.95)
                        #wd=None, wd_bn_bias=False, train_bn=True
                        #emb_szs=[],
                       )

### LR Finder

In [ ]:
lr = HpLrFinder(finder_type="fastai", which="steep", kwargs=...)

In [220]:
learn.fit_flat_cos(3, lr=lr)

epoch,train_loss,valid_loss,roc_auc_score,time
0,0.365408,0.357305,0.679694,00:02
1,0.359781,0.342277,0.746188,00:02
2,0.353049,0.334768,0.758574,00:02
